In [9]:
# 指定图片路径
image_path = 'F:\\JupyterWorkSpace\\BBBBBBS\\train\\c4\\img_14.jpg'  # 请确保这条路径是有效的

In [10]:
from keras.models import Model, load_model
from keras.layers import Input, Lambda, GlobalAveragePooling2D
from keras.applications import ResNet50, Xception, InceptionV3
from keras.preprocessing import image
import numpy as np
import h5py
import os

from tensorflow.keras.preprocessing import image  # 使用 tensorflow.keras

from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_v3_preprocess_input
from keras.applications.xception import Xception, preprocess_input as xception_preprocess_input

def load_custom_model(MODEL, weight_file, image_size):
    # 输入张量
    input_tensor = Input((*image_size, 3))
    base_model = MODEL(input_tensor=input_tensor, weights=None, include_top=False)
    
    # 加载权重
    base_model.load_weights("models/" + weight_file, by_name=True)
    
    # 构建最终模型
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    return model

def write_gap_test(tag, MODEL, weight_file, image_size, image_path, lambda_func=None, featurewise_std_normalization=True):
    # 检查图片路径是否有效
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return
    
    # 加载自定义模型
    model = load_custom_model(MODEL, weight_file, image_size)
    
    print(f"Predicting for image: {image_path}")
    
    # 直接加载和预处理单张图片
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 增加batch维度
    
    # 使用lambda函数进行预处理
    if lambda_func:
        img_array = lambda_func(img_array)
    elif featurewise_std_normalization:
        img_array = img_array / 255.0  # 假设图片值已归一化
    
    # 进行预测
    test = model.predict(img_array)
    print("Prediction result: {}".format(test.shape))

    # 将预测结果保存到文件
    output_dir = os.path.join("models", tag)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Begin creating database for {MODEL.__name__}")
    with h5py.File(os.path.join(output_dir, f"bottleneck_{MODEL.__name__}_test.h5"), 'w') as h:
        h.create_dataset("test", data=test)
    
    print(f"write_gap_test {MODEL.__name__} succeeded")

# 测试不同模型时指定的图片路径
# image_path = 'F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg'  # 确保这条路径是有效的

resnet50_weight_file = "resnet50-imagenet-finetune152.h5"
xception_weight_file = "xception-imagenet-finetune116.h5"
inceptionV3_weight_file = "inceptionV3-imagenet-finetune172.h5"

print("===== Test =====")
write_gap_test("Single", ResNet50, resnet50_weight_file, (240, 320), image_path)
write_gap_test("Single", Xception, xception_weight_file, (320, 480), image_path, xception_preprocess_input)


===== Test =====
Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 739ms/step
Prediction result: (1, 2048)
Begin creating database for ResNet50
write_gap_test ResNet50 succeeded
Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 571ms/step
Prediction result: (1, 2048)
Begin creating database for Xception
write_gap_test Xception succeeded


In [11]:
# 需要运行两次
write_gap_test("Single", InceptionV3, inceptionV3_weight_file, (320, 480), image_path, inception_v3_preprocess_input)

Predicting for image: F:/JupyterWorkSpace/BBBBBBS/train/c1/img_115.jpg
1/1 [==============================] - 1s 811ms/step
Prediction result: (1, 2048)
Begin creating database for InceptionV3
write_gap_test InceptionV3 succeeded


In [12]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import h5py

def extract_features_from_models(test_premodels, image_path, model_image_size):
    """
    Extract features for a single image from multiple pre-trained models.
    
    Args:
        test_premodels: List of pre-trained model file paths.
        image_path: Path to the image for prediction.
        model_image_size: Target size for image resizing.
    
    Returns:
        X_test: Extracted features from all pre-trained models.
    """
    X_test = []
    
    # 加载并预处理图像
    img = load_img(image_path, target_size=(model_image_size, model_image_size))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 增加批量维度

    # 从每个预训练模型中提取特征
    for model_file in test_premodels:
        with h5py.File(model_file, 'r') as h:
            features = np.array(h['test'])  # 提取预训练模型的特征
            X_test.append(features)

    # 合并所有特征
    X_test = np.concatenate(X_test, axis=1)
    return X_test

def predict_with_mixed_model(model, X_test):
    """
    Predict the class of a single image using the mixed model and extracted features.
    
    Args:
        model: Trained mixed model.
        X_test: Features extracted from pre-trained models.
    """
    # 使用混合模型进行预测
    y_pred = model.predict(X_test)
    # print(f"Prediction: {y_pred}")
    
    # 裁剪预测结果，确保在合理范围内
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print(f"Clipped Prediction: {y_pred}")
    
    print("------y_pred--------")
    print(y_pred)
    # 将 y_pred 扁平化为一维数组
    y_pred = y_pred.flatten()
    # 使用 np.argmax 找到最大概率值的索引
    predicted_class = np.argmax(y_pred)
    # 获取最大预测类别的概率值
    predicted_prob = y_pred[predicted_class]
    # 打印出预测的类别索引及其概率值
    print(f"Predicted class: c{predicted_class}, Probability: {predicted_prob}")

# 加载混合模型
model_mix = load_model("models/mixed-model.h5")

# 预训练模型文件路径
test_premodels = [
    "models/Single/bottleneck_ResNet50_test.h5",
    "models/Single/bottleneck_Xception_test.h5",
    "models/Single/bottleneck_InceptionV3_test.h5"
]


# 从预训练模型中提取特征
X_test = extract_features_from_models(test_premodels, image_path, 320)

# 使用混合模型进行预测
predict_with_mixed_model(model_mix, X_test)


1/1 [==============================] - 0s 32ms/step
Clipped Prediction: [[0.07775538 0.08730915 0.10445213 0.14501858 0.17224728 0.07249112
  0.08881389 0.07735401 0.0904341  0.08412421]]
------y_pred--------
[[0.07775538 0.08730915 0.10445213 0.14501858 0.17224728 0.07249112
  0.08881389 0.07735401 0.0904341  0.08412421]]
Predicted class: c4, Probability: 0.1722472757101059


# 汇总

In [6]:
# 指定图片路径
image_path = 'F:\\JupyterWorkSpace\\BBBBBBS\\test\\default_class\\img_2.jpg'  # 请确保这条路径是有效的
image_path = 'F:\\JupyterWorkSpace\\BBBBBBS\\train\\c1\\img_6.jpg'  # 请确保这条路径是有效的


from keras.models import Model, load_model
from keras.layers import Input, Lambda, GlobalAveragePooling2D
from keras.applications import ResNet50, Xception, InceptionV3
from keras.preprocessing import image
import numpy as np
import h5py
import os

from tensorflow.keras.preprocessing import image  # 使用 tensorflow.keras

from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_v3_preprocess_input
from keras.applications.xception import Xception, preprocess_input as xception_preprocess_input

def load_custom_model(MODEL, weight_file, image_size):
    # 输入张量
    input_tensor = Input((*image_size, 3))
    base_model = MODEL(input_tensor=input_tensor, weights=None, include_top=False)
    
    # 加载权重
    base_model.load_weights("models/" + weight_file, by_name=True)
    
    # 构建最终模型
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    return model

def write_gap_test(tag, MODEL, weight_file, image_size, image_path, lambda_func=None, featurewise_std_normalization=True):
    # 检查图片路径是否有效
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return
    
    # 加载自定义模型
    model = load_custom_model(MODEL, weight_file, image_size)
    
    print(f"Predicting for image: {image_path}")
    
    # 直接加载和预处理单张图片
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 增加batch维度
    
    # 使用lambda函数进行预处理
    if lambda_func:
        img_array = lambda_func(img_array)
    elif featurewise_std_normalization:
        img_array = img_array / 255.0  # 假设图片值已归一化
    
    # 进行预测
    test = model.predict(img_array)
    print("Prediction result: {}".format(test.shape))

    # 将预测结果保存到文件
    output_dir = os.path.join("models", tag)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Begin creating database for {MODEL.__name__}")
    with h5py.File(os.path.join(output_dir, f"bottleneck_{MODEL.__name__}_test.h5"), 'w') as h:
        h.create_dataset("test", data=test)
    
    print(f"write_gap_test {MODEL.__name__} succeeded")

resnet50_weight_file = "resnet50-imagenet-finetune152.h5"
xception_weight_file = "xception-imagenet-finetune116.h5"
inceptionV3_weight_file = "inceptionV3-imagenet-finetune172.h5"

print("===== Test =====")
write_gap_test("Single", ResNet50, resnet50_weight_file, (240, 320), image_path)
write_gap_test("Single", Xception, xception_weight_file, (320, 480), image_path, xception_preprocess_input)
# 需要运行两次
write_gap_test("Single", InceptionV3, inceptionV3_weight_file, (320, 480), image_path, inception_v3_preprocess_input)

import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import h5py

def extract_features_from_models(test_premodels, image_path, model_image_size):
    """
    Extract features for a single image from multiple pre-trained models.
    
    Args:
        test_premodels: List of pre-trained model file paths.
        image_path: Path to the image for prediction.
        model_image_size: Target size for image resizing.
    
    Returns:
        X_test: Extracted features from all pre-trained models.
    """
    X_test = []
    
    # 加载并预处理图像
    img = load_img(image_path, target_size=(model_image_size, model_image_size))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 增加批量维度

    # 从每个预训练模型中提取特征
    for model_file in test_premodels:
        with h5py.File(model_file, 'r') as h:
            features = np.array(h['test'])  # 提取预训练模型的特征
            X_test.append(features)

    # 合并所有特征
    X_test = np.concatenate(X_test, axis=1)
    return X_test

def predict_with_mixed_model(model, X_test):
    """
    Predict the class of a single image using the mixed model and extracted features.
    
    Args:
        model: Trained mixed model.
        X_test: Features extracted from pre-trained models.
    """
    # 使用混合模型进行预测
    y_pred = model.predict(X_test)
    # print(f"Prediction: {y_pred}")
    
    # 裁剪预测结果，确保在合理范围内
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print(f"Clipped Prediction: {y_pred}")
    
    print("------y_pred--------")
    print(y_pred)
    # 将 y_pred 扁平化为一维数组
    y_pred = y_pred.flatten()
    # 使用 np.argmax 找到最大概率值的索引
    predicted_class = np.argmax(y_pred)
    # 获取最大预测类别的概率值
    predicted_prob = y_pred[predicted_class]
    # 打印出预测的类别索引及其概率值
    print(f"Predicted class: c{predicted_class}, Probability: {predicted_prob}")

# 加载混合模型
model_mix = load_model("models/mixed-model.h5")

# 预训练模型文件路径
test_premodels = [
    "models/Single/bottleneck_ResNet50_test.h5",
    "models/Single/bottleneck_Xception_test.h5",
    "models/Single/bottleneck_InceptionV3_test.h5"
]


# 从预训练模型中提取特征
X_test = extract_features_from_models(test_premodels, image_path, 320)

# 使用混合模型进行预测
predict_with_mixed_model(model_mix, X_test)
print(image_path)

===== Test =====
Predicting for image: F:\JupyterWorkSpace\BBBBBBS\train\c1\img_6.jpg
1/1 [==============================] - 1s 657ms/step
Prediction result: (1, 2048)
Begin creating database for ResNet50
write_gap_test ResNet50 succeeded
Predicting for image: F:\JupyterWorkSpace\BBBBBBS\train\c1\img_6.jpg
1/1 [==============================] - 1s 827ms/step
Prediction result: (1, 2048)
Begin creating database for Xception
write_gap_test Xception succeeded
Predicting for image: F:\JupyterWorkSpace\BBBBBBS\train\c1\img_6.jpg
1/1 [==============================] - 1s 832ms/step
Prediction result: (1, 2048)
Begin creating database for InceptionV3
write_gap_test InceptionV3 succeeded
1/1 [==============================] - 0s 33ms/step
Clipped Prediction: [[0.09167638 0.08623686 0.08171918 0.152598   0.14745556 0.08306389
  0.15198664 0.04982068 0.0744195  0.08102331]]
------y_pred--------
[[0.09167638 0.08623686 0.08171918 0.152598   0.14745556 0.08306389
  0.15198664 0.04982068 0.0744195 